In [1]:
from IPython.display import FileLink, FileLinks

---
# Primeiro Passo - configuração do ambiente
---

Tutorial: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/get-set-up-for-amazon-ec2.html

## Criação de um usuário IAM (Instance and Access Management) (Lotufo que faz)

A Amazon não recomenda usar o login principal para esta finalidade, acredito que por razões de segurança da conta.

É recomendado criar usuários com poderes administrativos, que terão acesso a criação e uso de instâncias *dentro da conta principal*.

* Acessar: https://console.aws.amazon.com/iam/;
* **Users** -> **Add User**;
* Escolher username;
* Marcar a caixa "AWS Management Console access";
* Definir senha;
* Criar grupo e adicionar a adicionar a política: "AdministratorAccess", encontrada pela filtragem (ou adicionar ao grupo **rnpi**);
* Finalizar criação;
* Página de login será: https://your_aws_account_id.signin.aws.amazon.com/console/, onde your_aws_account_id é o ID da conta AWS.

## Criar um Key Pair

Para acessar a instância é necessário ter o arquivo da chave (.pem) salvo na máquina local. O único momento em que você consegue baixar este arquivo é na hora da criação do mesmo.

* Acessar: https://console.aws.amazon.com/ec2/;
* **Key Pairs** -> **Create Key Pair**;
* Definir um nome para o mesmo;
* Baixar o arquivo *.pem*;
* No terminal local digitar: ``chmod 400 <your_user_name-key-pair-region_name>.pem`` para que somente o seu usuário tenha permissão de acesso.

## Utilizar VPC padrão

Não sei bem o que faz, mas não parece ser necessário mexer nisso.

## Criar Security Group

Pelo que entendi é necessário liberar acesso SSH para cada IP que vá tentar conectar-se à instância.

Além disso adicionam-se algumas portas padrão e também adicionei a porta para acesso do Jupyter (que rodará através do Docker)

* Acessar: https://console.aws.amazon.com/ec2/;
* **Security Groups** -> **Add Security Groups**;
* Escolher um nome para o mesmo;
* Selecionar **Add Rule** e adicionar as seguintes regras:
    * Type: HTTP, Source: Anywhere (0.0.0.0/0)
    * Type: HTTPS, Source: Anywhere (0.0.0.0/0)
    * Type: SSH, Source: My IP (...) - *não recomendado liberar para todos os IPs*
    * Type: Custom TCP Rule, Port: ``<porta_jupyter>``, Source: Anywhere - *a porta padrão do Jupyter é a 8888*

---
# Segundo Passo - criação de uma instância

---

Tutorial: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EC2_GetStarted.html

* Acessar: https://console.aws.amazon.com/ec2/;
* Verificar se a região desejada já está selecionada (a princípio todas as máquinas estão sendo criadas em *US West (Oregon)*;
* Selecionar aba **Community AMIs** e procurar pela imagem:
    * Deep Learning AMI Ubuntu Linux - 1.0 (ami-85f276e5);
* Escolher o tipo de instância (g2.2xlarge para uso moderado de GPU);
* Selecionar **Review and Launch**;
* Selecionar **Edit security groups** -> **Select an existing security group** -> escolher grupo criado previamente;
* Selecionar **Launch**;
* Será necessário então criar ou escolher um *key pair*, caso já tenha sido previamente criado;
* Selecionar o *key pair* e marcar a caixa de seleção;
* Selecionar **Launch instance**.

---
# Terceiro Passo - acesso à uma instância
---

* Acessar: https://console.aws.amazon.com/ec2/;

## Acesso através de SSH - Terminal local

Após criar a sua instância, clique no link **View Intsances**. Verifique se a sua instância está no estado **Running**. Selecione o box da sua instância e clique em **Connect**. Aparecerão as instruções para se conectar via ssh na instância.

O comando para acesso segue o padrâo abaixo:

* ``ssh -i "<path_arquivo_chave_pem>" ubuntu@<dns_público_instância>``

**Atenção**: Caso não consiga se conectar por SSH. Se o erro for simplesmente `timeout`. O problema pode ser o IP não estar autorizado. Neste caso é preciso voltar ao Security Group e adicionar o acesso ao novo IP, da mesma forma que feito anteriormente.


---
# Quarto Passo - Setup do ambiente
---



O setup do ambiente consiste em instalar:

docker: https://docs.docker.com/engine/installation/linux/ubuntu/

nvidia-docker: https://github.com/NVIDIA/nvidia-docker

Para realização do setup basta executar o seguinte script na instância da AWS: [AWS Setup Script](../others/AWS_setup.sh)

Faça o download para sua máquina e copie o script para AWS utilizando o seguinte comando scp:

* ``scp -i "<path_arquivo_chave_pem>" maquina@<path_do_script> ubuntu@<dns_público_instância>:/home/ubuntu``

**Obs. 1:** Ao executar o script, durante a configuraçãom aparecerá a imagem abaixo, clique em enter para continuar o setup.


<img src="../figures/tela.png" width="700" height="800" />

**Obs. 2**: Verificar se o plugin do nvidia-docker está funcionando:

* ``service nvidia-docker status``

Caso não esteja mesmo rodando é preciso inicializá-lo com:

* ``sudo service nvidia-docker start``

**Obs. 3:** Teste do nvidia-docker, deverá baixar a imagem e ao ser executado, encontrar a(s) placa(s) de vídeo da máquina:
* ``sudo nvidia-docker run --rm nvidia/cuda nvidia-smi``

---
# Quinto Passo - rodando o Jupyter através do Docker
---

Após o setup do ambiente, para utilizar o jupyter pro meio do docker no AWS, basta executar os seguintea comandos:

* ``nvidia-docker run --rm --name jupyter -it -p 8888:8888 -v /home/ubuntu/notebooks:/opt/projects \
-v /home/ubuntu/datasets:/opt/datasets rcm2016/keras:1.0``

  * As pastas notebooks e datasets serão criadas no /home/ubuntu. Todo o conteúdo dessas pastas é compatilhado com as pasatas datasets e projects dentro do docker.

Para acessar o jupyter digite no browser:

* ``<dns_público_instância>:8888``
   * O token para acessar o notebook aparece no terminal. Utilize-o para logar no jupyter-notebook.






---
# Informações adicionais
---

## Informações gerais

* Os dados colocados nas instâncias EC2 são persistidos sem problemas no armazenamento EBS.
* Enquanto o container executado no Docker existe, os dados são persistidos, mesmo que sua execução seja parada (``docker stop <nome_instancia_container>``) ou a instância da Amazon seja desligada (neste caso o container também é parado, então ao ligar a instância é necessário iniciá-lo, da mesma forma (``docker start <nome_instancia_docker>``).
* De qualquer forma, parece mais interessante garantir que os dados sejam persistidos fora do Docker, caso ocorra algum problema e o container precise ser instanciado novamente.
* Se possível, seria interessante inicializar a instância do AWS e salvar os dados (baixar do repositório, por exemplo) a serem utilizados no curso antes de conectar-se ao Docker, pois a partir daí o mesmo poderia ser iniciado uma única vez, já montando a pasta na instância do AWS dentro da instância do Docker.
* Em todo caso, se o material já vier na imagem do Docker, basta inicializar uma instância do Docker, copiar o conteúdo para a instância do AWS (``docker cp <nome_instancia_container>:<diretório_docker> <diretório_aws>``).


## Informações sobre o serviço ECS (EC2 Container Service)

* Uma maneira de automatizar um pouco a execução de instâncias do Docker.
* Não dá a opção de usar o tipo de instância ``g2.2xlarge``, que usa GPU.
* É necessário criar um cluster, um serviço e uma tarefa, mas isto não é tão complicado.
* Parece que ao matar uma instância o serviço automaticamente cria outra no lugar, então foi um pouco chato de remover as instâncias.
* Precisaria ver a questão de copiar os dados para a instância AWS para que ao inicializar a instância do Docker a mesma já pudesse montar os diretórios do curso corretamente.